In [1]:
### lHC-MR simulation script 
## Generating n data simulations
## Input: set up the parameters corresponding to the scenario, bolean for more complex scenarios,
# boolean indicating detailed saving or not
## Output: betXY, thS (parameters), sig1 + pi1 (local LD structure), nX, nY, m0 (number of SNPs in local region), 
# piU (constant), if desired: hrX + hrU + hrY

library(data.table)
library(rhdf5)
library(lhcMR)
library(dplyr)
library(PearsonDS)



Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:

#### start to change
## set working directory
#setwd("/data/sgg3/liza/SEM_Sim/R_trans")

## scenario number
dS = 1;
## data generations for this scenario
nsimi = 1
## boolean set up to indicate detailed save (with hrX/U/Y), kurtosis (non-normal scenario), or presence of two confounders (twoU)
detailedSave = FALSE
kurtosis = FALSE #if true change kurtosis level in first if clause below
twoUs = FALSE #if true change two U properties in firs if clause below
#### end to change

## read in pi1 and sig1 + R matrix
Rmat = fread("../LD_vsGM2.txt")
c1 = fread("../pi1-sigma1_vsGM2.txt") #234000 x 5001

pi1 = unlist(c1[,1]) # proportion of non-zero component of rho
sig1 = unlist(c1[,2]) # SD of non-zero component of rho
sim_num = length(sig1);  # number of genotyped SNPs
m0 = 5e3+1;  # n_local true sequence variants in each SNP region - used for pi1/sig1
rm(c1)#to save space

In [3]:


#### start to change
## create bX bY from these parameters
piX = 5e-3;
piU = 5e-2;
piY = 1e-2;
h2X = .25;
h2U = .3;
h2Y = .2;
qX = .3;
qY = .2;
alp = .3;
bet = 0;
M = 1e7;
sigX = sqrt(h2X/(M*piX));
sigU = sqrt(h2U/(M*piU));
sigY = sqrt(h2Y/(M*piY));
tX = qX*sqrt(h2U);
tY = qY*sqrt(h2U);
nX = 5e5;
nY = 5e5;
iX = 1;
iY = 1;
iXY = 0;

if(kurtosis){
  krt = 5; #2 5 10
}
if(twoUs){
  qX1 = .3;
  qY1 = .2;
  qX2 = .4;
  qY2 = .3;
  tX1 = qX1*sqrt(h2U);
  tY1 = qY1*sqrt(h2U);
  tX2 = qX2*sqrt(h2U);
  tY2 = qY2*sqrt(h2U);
}
#### end to change

In [6]:

  simi = 2

system("mkdir data-mat")
  message(simi)
  hX =  matrix(rbinom(sim_num*m0,1,piX),ncol=m0); #binornd(1,piX,sim_num,m0);
  seli = which(hX==1); #find(hX==1);
  tmp = matrix(rnorm(length(seli),0,sigX),ncol=1); #normrnd(0,sigX,length(seli),1);
  hX[seli] = tmp;
  if(kurtosis){
    hX = matrix(rbinom(sim_num*m0,1,piX),ncol=m0);
    sv1 = which(hX==1);
    tmp3 = matrix(PearsonDS::rpearson(length(sv1),moments=c(mean=0,variance=sigX,skewness=0,kurtosis=kurt)),ncol=1);#pearsrnd(0,sigX,0,krt,length(sv1),1);
    hX[sv1] = tmp3;
  }
  
  hU =  matrix(rbinom(sim_num*m0,1,piU),ncol=m0); #binornd(1,piU,sim_num,m0);
  seli = which(hU==1); #find(hX==1);
  tmp = matrix(rnorm(length(seli),0,sigU),ncol=1); #normrnd(0,sigU,length(seli),1);
  hU[seli] = tmp;
  if(kurtosis){
    hU = binornd(1,piU,sim_num,m0);
    sv1 = find(hU==1);
    tmp3 = pearsrnd(0,sigU,0,krt,length(sv1),1);
    hU(sv1) = tmp3;
  }
  if(twoUs){
    hU1 = matrix(rbinom(sim_num*m0,1,piU),ncol=m0);
    seli = which(hU1==1);
    tmp = matrix(rnorm(length(seli),0,sigU),ncol=1)
    hU1[seli] = tmp;
    hU2 = matrix(rbinom(sim_num*m0,1,piU),ncol=m0);
    seli = which(hU2==1);
    tmp = matrix(rnorm(length(seli),0,sigU),ncol=1)
    hU2[seli] = tmp;
  }
  
  hY =  matrix(rbinom(sim_num*m0,1,piY),ncol=m0); #binornd(1,piY,sim_num,m0);
  seli = which(hY==1); #find(hX==1);
  tmp = matrix(rnorm(length(seli),0,sigY),ncol=1); #normrnd(0,sigY,length(seli),1);
  hY[seli] = tmp;
  if(kurtosis){
    hY = binornd(1,piY,sim_num,m0);
    sv1 = find(hY==1);
    tmp3 = pearsrnd(0,sigY,0,krt,length(sv1),1);
    hY(sv1) = tmp3;
  }
  
  hrX = rowSums(Rmat*hX); #sum(R.*hX,2);
  hrU = rowSums(Rmat*hU); #sum(R.*hU,2);
  hrY = rowSums(Rmat*hY); #sum(R.*hY,2);
  if(twoUs){
    hrU1 = rowSums(Rmat*hU1); #sum(R.*hU1,2);
    hrU2 = rowSums(Rmat*hU2); #sum(R.*hU2,2);
  }
  ## Noise is generated from a variance-covariance matrix (X LD-score intercept divided by nX, cross LD score intercept rescaled by dividing by sqrt of nX*nY, and Y LD score intercept divided by nY).
  R0 = matrix(c(iX/nX,iXY/sqrt(nX*nY),iXY/sqrt(nX*nY),iY/nY),ncol=2,nrow=2, byrow = T) # [iX/nX,iXY/sqrt(nX*nY);iXY/sqrt(nX*nY),iY/nY];
  C = chol(R0);
  noi = matrix(rnorm(sim_num*2,0,1),ncol=2) #normrnd(0,1,sim_num,2);
  noi = noi %*% C #noi*C;
  
  ## Generate effects
  betX = (hrX + ((bet*qY)+qX)*hrU + bet*hrY)/(1-(alp*bet)) + noi[,1] #(hrX+(bet*qY+qX)*hrU+bet*hrY)/(1-alp*bet)+noi(:,1);
  betY = (alp*hrX + ((alp*qX)+qY)*hrU + hrY)/(1-(alp*bet)) + noi[,2] #(alp*hrX+(alp*qX+qY)*hrU+hrY)/(1-alp*bet)+noi(:,2);
  if(twoUs){
    betX = hrX + ((bet*qY1)+qX1)*hrU1 + ((bet*qY2)+qX2)*hrU2 + bet*hrY + noi[,1];
    betY = alp*hrX + ((alp*qX1)+qY1)*hrU1 + ((alp*qX2)+qY2)*hrU2 + hrY + noi[,2];
  }
  betXY = cbind(betX,betY) # [betX,betY];
  
  #save parameters set up
  thS = c(piX,piY,sigX,sigY,tX,tY,alp,bet,iX,iY,iXY)
  if(twoUs){
    thS = c(piX,piY,sigX,sigY,tX1,tY1,alp,bet,iX,iY,iXY)
  }
  if(detailedSave){
    if(twoUs){
      save(betXY, hrX, hrU1, hrU2, hrY, thS, sig1, pi1, nX, nY, m0, piU, qX1, qX2, qY1, qY2,
           file = paste0("./data-mat/exampleDataB",dS,"_sim",simi,".RData"))
    }else{
      save(betXY, hrX, hrU, hrY, thS, sig1, pi1, nX, nY, m0, piU, file = paste0("./data-mat/exampleDataB",dS,"_sim",simi,".RData"))
    }
  }else{
    if(twoUs){
      save(betXY, thS, sig1, pi1, nX, nY, m0, piU, qX1, qX2, qY1, qY2, file = paste0("./data-mat/exampleDataB",dS,"_sim",simi,".RData"))
    }else{
      save(betXY, thS, sig1, pi1, nX, nY, m0, piU, file = paste0("./data-mat/exampleDataB",dS,"_sim",simi,".RData"))
    }
  }


2



In [3]:
library(rslurm)


In [4]:
## parameter set up
dS = 1 #scenario number
nsimi = 1 #number of data generations from previous scripts
nSP = 5 #number of starting points for each generation
saveSP = TRUE #to save the starting points used for each data generation
paral_method = "lapply" #"lapply"
#### end to change

## single trait analysis to estimate pi, h2, i, followed by generating 50 SP for each data generation and optimisation
SP = 3; #number of starting points for single trait analysis to estimate pi, h2 and i
M = 1e7; #number of SNPs
nCores=1; #to set up for optimisation in lapply

In [5]:
simi = 2
load(paste0("./data-mat/exampleDataB",dS,"_sim",simi,".RData")) 
  wd = rep(1, length(sig1)) #ones(length(sig1),1);
  piX = thS[1];
  piY = thS[2];
  sigX = thS[3];
  sigY = thS[4];
  tX = thS[5];
  tY = thS[6];
  alp = thS[7];
  bet = thS[8];
  iX = thS[9];
  iY = thS[10];
  iXY = thS[11];
  h2X = piX*M*sigX^2;
  h2Y = piY*M*sigY^2;
  herX = h2X+tX^2;
  herY = h2Y+tY^2;

In [6]:
print(dim(betXY))

[1] 234225      2


In [7]:
## SP calculation
  sp_piX = runif(SP,0.00001,0.1)
  sp_h2X = runif(SP,0.15,0.5)
  sp_iX = runif(SP,0.5,1.5)
  
  # Get piX, iX (and total_h2) in a separate step - single trait analysis for X and Y
  para=cbind(sp_piX,sp_h2X,sp_iX)
  sp_mat=matrix(unlist(para), ncol=SP, byrow = FALSE)
  colnames(sp_mat)=colnames(para)
  par.df = data.frame(par=I(apply(sp_mat,1,as.list))) #generate a dataframe of lists for each row of parameters - input for rslurm/lapply
  

In [8]:
print(par.df)

           par
1 0.012427....
2 0.023595....
3 0.055882....


In [9]:
print(betXY)

                   betX          betY
     [1,] -1.058761e-03  6.876877e-04
     [2,]  8.248522e-04 -2.305306e-03
     [3,]  1.560332e-03  5.317878e-04
     [4,] -1.632751e-03 -1.668480e-03
     [5,] -1.072870e-03  3.987367e-04
     [6,] -3.637441e-03 -3.613718e-03
     [7,] -5.116208e-03 -6.368242e-03
     [8,]  9.563875e-05 -1.611130e-03
     [9,] -5.575484e-03 -9.810052e-04
    [10,] -1.931774e-03 -1.353314e-03
    [11,] -4.831203e-03  3.444184e-03
    [12,]  2.340473e-03  2.888741e-03
    [13,]  3.283622e-04  2.895978e-03
    [14,] -4.873775e-04 -3.438776e-03
    [15,] -4.683650e-03 -4.372981e-03
    [16,] -7.234728e-04 -1.984378e-03
    [17,] -1.782298e-05  4.063701e-03
    [18,] -1.077629e-02 -3.616565e-03
    [19,] -2.402757e-04  1.965892e-03
    [20,]  7.615487e-04  5.774549e-04
    [21,] -1.737002e-03  3.152628e-03
    [22,] -1.760247e-03  9.098380e-04
    [23,]  6.226178e-03 -3.731249e-03
    [24,]  2.635745e-04 -2.393340e-04
    [25,] -9.760115e-04 -2.482417e-03
    [26,]  2

In [10]:
  test.exp <- parallel::mclapply(par.df[[1]], function(x) {
    theta = unlist(x) #theta = unlist(par.df[[1]][1])
    test1 = optim(theta, lhcMR:::singleTrait_likelihood,
                  betX=betXY[,1], pi1=pi1, sig1=sig1, w8s=wd, M=M,
                  m0=m0, nX=nX, bn=2^7, bins=10,
                  method = "Nelder-Mead",
                  control = list(maxit = 5e3))
    
    list("mLL"=test1$value,"par"=test1$par,"conv"=test1$convergence)
  }, mc.cores = nCores)
  test.exp = as.data.frame(t(matrix(unlist(test.exp), nrow=length(unlist(test.exp[1])))))
  

In [11]:
print(test.exp)

        V1           V2        V3       V4 V5
1 -1090236 -0.001962153 0.2648030 1.054398  0
2 -1090236  0.001962214 0.2652648 1.051970  0
3 -1090236 -0.001960311 0.2651396 1.053560  0


In [12]:
  test.out <- parallel::mclapply(par.df[[1]], function(x) {
    theta = unlist(x)
    test2 = optim(theta, lhcMR:::singleTrait_likelihood,
                  betX=betXY[,2], pi1=pi1, sig1=sig1, w8s=wd, M=M,
                  m0=m0, nX=nY, bn=2^7, bins=10,
                  method = "Nelder-Mead",
                  control = list(maxit = 5e3))
    
    list("mLL"=test2$value,"par"=test2$par,"conv"=test2$convergence)
  }, mc.cores = nCores)
  test.out = as.data.frame(t(matrix(unlist(test.out), nrow=length(unlist(test.out[1])))))
  

In [13]:
print(test.out)

        V1           V2        V3       V4 V5
1 -1095826 -0.004293567 0.2355313 1.069502  0
2 -1095826 -0.004261942 0.2352531 1.071263  0
3 -1095826 -0.004249483 0.2352188 1.071533  0


In [14]:
  colnames(test.exp)=c("mLL","piX", "h2X","iX","conv")
  colnames(test.out)=c("mLL","piX", "h2X","iX","conv")
  res_exp_min = test.exp[which(test.exp$mLL == min(test.exp$mLL)), ]
  res_exp = abs(res_exp_min[2:4]) #piX,h2X,iX
  res_out_min = test.out[which(test.out$mLL == min(test.out$mLL)), ]
  res_out = abs(res_out_min[2:4]) #piY,h2Y,iY
  
  pi_X = as.numeric(res_exp[1])
  pi_Y = as.numeric(res_out[1])
  h2_x = as.numeric(res_exp[2]) #total heritability, not used
  h2_y = as.numeric(res_out[2]) #total heritability, not used
  i_X = as.numeric(res_exp[3])
  i_Y = as.numeric(res_out[3])
  
  # Generate the rest of the starting points 
  sp_tX = runif(nSP,0,0.5)
  sp_tY = runif(nSP,-0.5,0.5)
  sp_h2X = max(0,h2_x-(sp_tX^2))
  sp_h2Y = max(0,h2_y-(sp_tY^2))
  sp_axy = replicate(nSP, (runif(1,-0.5,0.5)))
  sp_ayx = replicate(nSP, (runif(1,-0.5,0.5)))
  sp_iXY = replicate(nSP, (iXY+runif(1,-0.05,0.05))) #rep(i_XY,50)

In [15]:
print(res_exp_min)

       mLL         piX       h2X      iX conv
2 -1090236 0.001962214 0.2652648 1.05197    0


In [16]:
  para=cbind(sp_h2X,sp_h2Y,sp_tX,sp_tY,sp_axy,sp_ayx,sp_iXY)
  sp_mat=matrix(unlist(para), ncol=7, byrow = FALSE)
  colnames(sp_mat)=c("sp_h2X","sp_h2Y","sp_tX","sp_tY","sp_axy","sp_ayx","sp_iXY")
  
  SP_list = list("iX"=i_X,"iY"=i_Y,"piX"=pi_X,"piY"=pi_Y,"sp_mat"=sp_mat)
  if(saveSP){
    save(SP_list, file = paste0("./data-mat/exampleDataB",dS,"_",simi,"_sp",nSP,".RData"))
  }
  
  ## optimisation of said set of starting points
  bn = 2^7
  bins = 10
  param="comp" #possibility to develop nesting in later version
  w8s=wd #must be the same name as in likelihood function
  iX=i_X;iY=i_Y;piX=pi_X;piY=pi_Y #must be the same name as in likelihood function
  parscale2 = c(1e-1,1e-1,1e-1,1e-1,1e-1,1e-1,1e-1)
  nCores = NA #for lapply: update to number wanted, or leave as NA to use third of available cores
  if(is.na(nCores)){
    nCores = max(1,floor((parallel::detectCores())/3))
  } else {
    if(nCores > parallel::detectCores()){
      cat(print("Core number chosen is greater than cores available\n"))
      stop()
    }
  }

In [17]:
  # Generate a dataframe of lists for each row of parameters - input for rslurm/lapply
  par.df = data.frame(par=I(apply(sp_mat,1,as.list)))
  

In [91]:
pairTrait_twoStep_likelihood <- function(theta,betXY,pi1,sig1,w8s,m0,pi_U=0.1,pi_X,pi_Y,i_X,i_Y,nX,nY,model="comp",bn=2^8,bins=15,M=1e7){

  M = M #1e7
  piX = pi_X
  piU = pi_U #piU is unidentifiable
  piY = pi_Y
  h2X = abs(theta[1]);
  h2Y = abs(theta[2]);
  iX = i_X;
  iY = i_Y;

  if(model=="comp"){
    tX = abs(theta[3]);
    tY = theta[4];
    axy = theta[5];
    ayx = theta[6];
    iXY = theta[7];
  }

  # if(model=="axy"){
  #   tX = abs(theta[3]);
  #   tY = theta[4];
  #   axy = 0;
  #   ayx = theta[5];
  #   iXY = theta[6];
  # }
  #
  # if(model=="ayx"){
  #   tX = abs(theta[3]);
  #   tY = theta[4];
  #   axy = theta[5];
  #   ayx = 0;
  #   iXY = theta[6];
  # }
  #
  # if(model=="tX"){
  #   tX = 0;
  #   tY = theta[3];
  #   axy = theta[4];
  #   ayx = theta[5];
  #   iXY = theta[6];
  # }
  #
  # if(model=="tY"){
  #   tX = abs(theta[3]);
  #   tY = 0;
  #   axy = theta[4];
  #   ayx = theta[5];
  #   iXY = theta[6];
  # }

  if(model=="U"){
    tX = 0;
    tY = 0;
    axy = theta[3];
    ayx = theta[4];
    iXY = theta[5];
  }


  sigX = sqrt(h2X/(piX*M))
  sigU = sqrt(1/(piU*M)); #h2U = 1
  sigY = sqrt(h2Y/(piY*M))

  m = nrow(betXY)

  Rp = matrix(c(iX/nX,iXY/sqrt(nX*nY),iXY/sqrt(nX*nY),iY/nY), nrow=2, ncol=2, byrow = T)

  bX = array(betXY[,1], c(1,1,m))
  bY = array(betXY[,2], c(1,1,m))

  minX = mean(bX)-(5*sd(bX));
  maxX = mean(bX)+(5*sd(bX));
  minY = mean(bY)-(5*sd(bY));
  maxY = mean(bY)+(5*sd(bY));

  dX = (maxX-minX)/(bn-1);
  dY = (maxY-minY)/(bn-1);

  minX = minX-dX/2;
  maxX = maxX+dX/2;
  minY = minY-dY/2;
  maxY = maxY+dY/2;

  bXi = ceiling((bX-minX)/dX);
  bYi = ceiling((bY-minY)/dY);

  bXi[bXi<1] = 1;
  bXi[bXi>bn] = bn;
  bYi[bYi<1] = 1;
  bYi[bYi>bn] = bn;

  if(max(c(piX,piU,piY)) > 0.2 || min(c(piX,piU,piY)) < 1e-6 || max(c((h2X+tX^2),(h2Y+tY^2)))>1
     || min(c((h2X+tX^2),(h2Y+tY^2))) < 1e-6|| abs(axy)>=1 || abs(ayx)>=1 || min(c(iX,iY))<=0.5
     || max(c(iX,iY))>=1.5 || abs(iXY)>1){
    logL = 1e10
    nmiss = 0
  }else{
    min_pi1 = min(pi1)-1e-10;
    max_pi1 = max(pi1)+1e-10;
    dp = (max_pi1-min_pi1)/bins;
    pc = min_pi1 + (dp * matrix(seq(0.5,(bins-0.5),1),ncol=1))
    pix = ceiling((pi1-min_pi1)/dp);
    min_sig1 = min(sig1)-1e-10;
    max_sig1 = max(sig1)+1e-10;
    ds = (max_sig1-min_sig1)/bins;
    sc = min_sig1 + (ds * matrix(seq(0.5,(bins-0.5),1),ncol=1))
    six = ceiling((sig1-min_sig1)/ds);
    cix = pix + bins*(six-1);
    uni_cix = sort(unique(cix))
    ucix = match(uni_cix, cix)
    ixMap = match(cix, uni_cix)
    Sig1 = sc[six[ucix]];
    Pi1 = pc[pix[ucix]];
    mm = length(Sig1);

    sigK = aperm(array(rep(Sig1, bn*mm), c(mm,bn,bn)), c(2,3,1))
    piK = aperm(array(rep(Pi1, bn*mm), c(mm,bn,bn)), c(2,3,1))

    j = (0:(bn-1))
    vi = 2*pi*(j-bn/2)/(maxX-minX-dX);
    wj = 2*pi*(j-bn/2)/(maxY-minY-dY);
    Rx = array(rep(vi, bn*mm), dim = c(bn, bn, mm))
    Ry = aperm( array(rep(wj, bn*mm), dim = c(bn, bn, mm)),c(2,1,3))


    coX = (Rx+axy*Ry)*sigK;
    coY = (Ry+ayx*Rx)*sigK;
    coU = sigU * ((tX+ayx*tY)*Rx + (tY+axy*tX)*Ry) * sigK;

    Lx = -m0*piX*( 1 - 1/sqrt(1+sigX^2*coX^2) )*piK;
    Ly = -m0*piY*( 1 - 1/sqrt(1+sigY^2*coY^2) )*piK;
    Lu = -m0*piU*( 1 - 1/sqrt(1+coU^2) )*piK;
    Le = -(1/2)*( (Rp[1,1]*Rx^2+Rp[2,2]*Ry^2) + 2*Rp[1,2]*(Rx*Ry) );

      # /!\ complex numbers here!
    mf_init = -2*log(as.complex(-1)) *
      ( matrix( rep( ((minX+dX/2)/(maxX-minX-dX))*j, length(j) ), nrow = length(j) , byrow=T) +
          ( matrix( rep( ((minY+dY/2)/(maxY-minY-dY))%*%t(j), length(j) ),  nrow = length(j) ) ) )
      
    mf = array(rep(mf_init, mm), dim=c(bn, bn, mm))
    phi = exp(Lx+Ly+Lu+Le+mf);

    FFT=array(NA, dim=c(bn, bn, mm))
    for(l in 1:mm){
      FFT[,,l] = fft(phi[,,l])
    }

    FFTmod_init = (1/((maxX-minX-dX)*(maxY-minY-dY)))*(as.complex(-1))^(bn*((minX+dX/2)/(maxX-minX-dX) +
                                                                              (minY+dY/2)/(maxY-minY-dY)) + (outer(j,j,FUN="+")) )


    FFTmod = array(rep(FFTmod_init, mm), dim=c(bn,bn,mm));
    FFT0 = Re(FFT*FFTmod);
    #print(FFT0)
    ixF = cbind(bXi, bYi, ixMap);
    pfE = FFT0[ixF];

    selu = which(pfE>0);
    pfE = pfE[selu];
    my_w8s = w8s[selu]  #update weights vector if some SNPs are excluded
    lpfE = log(pfE);
    logL = -m * mean(log(pfE*my_w8s))
  }
  print(logL)
  return(logL)
}

In [92]:
temp_theta = unlist(par.df[[1]][[1]])
print(temp_theta)
# Start timing
timing <- system.time({
  out <- optim(temp_theta, pairTrait_twoStep_likelihood, betX=betXY, pi1=pi1, sig1=sig1, w8s=w8s, M=M,
               m0=m0, nX=nX, nY=nY, pi_U=piU, pi_X=piX, pi_Y=piY, i_X=iX, i_Y=iY,
               bn=2^7, bins=10, method = "Nelder-Mead",
               control = list(maxit = 5000, parscale = parscale2))
})

# Print the timing
print(timing)
print(out)

     sp_h2X      sp_h2Y       sp_tX       sp_tY      sp_axy      sp_ayx 
 0.20297614  0.22687459  0.33887307 -0.37550837  0.48169111 -0.48913198 
     sp_iXY 
 0.03350195 
, , 1

             [,1]       [,2]       [,3]       [,4]       [,5]       [,6]
  [1,] 0.05876823 0.05876823 0.05876823 0.05876823 0.05876823 0.05876823
  [2,] 0.05876823 0.05876823 0.05876823 0.05876823 0.05876823 0.05876823
  [3,] 0.05876823 0.05876823 0.05876823 0.05876823 0.05876823 0.05876823
  [4,] 0.05876823 0.05876823 0.05876823 0.05876823 0.05876823 0.05876823
  [5,] 0.05876823 0.05876823 0.05876823 0.05876823 0.05876823 0.05876823
  [6,] 0.05876823 0.05876823 0.05876823 0.05876823 0.05876823 0.05876823
  [7,] 0.05876823 0.05876823 0.05876823 0.05876823 0.05876823 0.05876823
  [8,] 0.05876823 0.05876823 0.05876823 0.05876823 0.05876823 0.05876823
  [9,] 0.05876823 0.05876823 0.05876823 0.05876823 0.05876823 0.05876823
 [10,] 0.05876823 0.05876823 0.05876823 0.05876823 0.05876823 0.05876823
 [11,] 0.05876823 

Timing stopped at: 1.266 0 1.266



In [20]:
print(out)

$par
     sp_h2X      sp_h2Y       sp_tX       sp_tY      sp_axy      sp_ayx 
 0.20297614  0.22687459  0.33887307 -0.32659517  0.48169111 -0.48913198 
     sp_iXY 
 0.03350195 

$value
[1] -2168023

$counts
function gradient 
      10       NA 

$convergence
[1] 1

$message
NULL



In [21]:
log.file <- "parallel_log.txt"


In [22]:
    cat(print("Running optimisation"))
    test.res <- parallel::mclapply(par.df[[1]], function(x) {
      theta = unlist(x)
      cat("Something\n", file=log.file, append=TRUE)  # This writes to a file instead of the console
      test = optim(theta, pairTrait_twoStep_likelihood,
                   betX=betXY, pi1=pi1, sig1=sig1, w8s=w8s, M=M,
                   m0=m0, nX=nX, nY=nY, pi_U=piU, pi_X=piX, pi_Y=piY, i_X=iX, i_Y=iY,
                   bn=2^7, bins=10,
                   method = "Nelder-Mead",
                   control = list(maxit = 5e3,
                                  parscale = parscale2))
      
      list("mLL"=test$value,"par"=test$par,"conv"=test$convergence)
    }, mc.cores = nCores)
    
    test.res = as.data.frame(t(matrix(unlist(test.res), nrow=length(unlist(test.res[1])))))
    colnames(test.res)=c("mLL", "h2X","h2Y","tX","tY","axy","ayx","iXY","conv")
    
    test.res %>%
      dplyr::mutate(h2X = abs(h2X),
                    h2Y = abs(h2Y),
                    tX = abs(tX)) -> res_values
    
    res_values = cbind("SP"=c(1:nrow(res_values)),"mLL"=res_values[,1],"piX"=piX,"piY"=piY,res_values[,-1],"iX"=iX,"iY"=iY)
    #write.csv(res_values, paste0("FullResult_DataB",dS,"_",simi,".csv"), row.names = FALSE)
    
    res_est = res_values[which(res_values$mLL==min(res_values$mLL)),]
    res_est = dplyr::select(res_est, -c(SP,mLL,conv))
    #write.table(res_est, paste0("SummaryResult_DataB",dS,"_",simi,".csv"), sep = ",", row.names = FALSE, col.names = FALSE, append = TRUE)

[1] "Running optimisation"
Running optimisation

In [ ]:
print(res_est)

In [ ]:
print(res_values)

In [ ]:
print(pi_X)